In [1]:
import json
import pandas as pd
import numpy as np
import os 
from os import listdir
from os.path import isfile, join
from datetime import datetime

In [2]:
dir='C:\\Users\hecto\PreWorks\Ironhack_labs\Slack_FP\Slack/'
fol = os.listdir(dir)
print(fol)

['berlin-get-together', 'channels.json', 'data-memes', 'data-resources', 'data-viz-beauties', 'dataqueens', 'flo_standup_group_week5', 'general', 'getting_started_tips', 'integration_logs.json', 'katas', 'lab-questions', 'music', 'project-questions', 'random', 'room4', 'team-project-1', 'users.json']


In [3]:
dchannels=[file for file in fol if file.endswith('json')]
channels=list(set(fol)-set(dchannels))
channels

['berlin-get-together',
 'data-viz-beauties',
 'flo_standup_group_week5',
 'room4',
 'general',
 'music',
 'dataqueens',
 'data-memes',
 'getting_started_tips',
 'project-questions',
 'data-resources',
 'random',
 'katas',
 'team-project-1',
 'lab-questions']

In [4]:
#jfiles = [f for f in listdir('slack\general') if isfile(join('slack\general', f))]
#jfiles

In [24]:
date_time = []
user = []
text = []
file_content = []
reactions = []
reactions_user=[]
replies_user = []
reply_users=[]
channel=[]
replies_time=[]
link=[]
file_url=[]
for c in channels:
    files = [f for f in listdir("slack/" +c) if isfile(join("slack/" +c, f))]

    for file in files:
        data=json.load(open( "slack/"+c +"/" + file,encoding='utf-8'))

        for message in data:
            channel.append(c)
            date_time.append(datetime.fromtimestamp(float(message["ts"])))
            if 'files' in message.keys():
                for i in message['files']:
                    file_url.append(i['url_private_download'])
            else:
                file_url.append(0)
            if 'attachments' in message.keys():
                if 'title_link' in 'attachments':
                    for i in message['attachments']:
                        link.append(i['title_link'])
                else:
                    link.append(0)

In [26]:
len(file_url)

2631

In [25]:
len(date_time)

2613

In [ ]:
if 'attachments' in message.keys():
                if 'title_link' in 'attachments':
                    
                    link.append(message['attachments'][0]['title_link'])
                else:
                    link.append(0)
            else:
                link.append(0)
            if 'reply_users_count' in message.keys():
                reply_users.append(message['reply_users_count'])
            else: 
                reply_users.append(0)
                    
            if 'replies' in message.keys():
                replies_time.append(message['replies'][0]['ts'])
                replies_user.append(message['replies'][0]['user'])
            else: 
                replies_time.append(0)
                replies_user.append(0)
            
            if 'reactions' in message.keys():
                count=0
                for reaction in message["reactions"]:
                    count=count+reaction['count']
                reactions.append(count)
            else: 
                reactions.append(0)
            if 'reactions' in message.keys():
                reactions_user.append(message['reactions'][0]['users'][0])
            else:
                reactions_user.append(0)

In [6]:
df_final = pd.DataFrame({"daytime":date_time,
                        "user": user,'text':text,'file_content':file_content,'reactions':reactions,
                        'replies_time':replies_time,'channel':channel,'reply_users':reply_users,'replies_user':replies_user,
                        'file_url':file_url,'link':link,
                        'reactions_user':reactions_user})

In [7]:
df_final['daytime']=pd.to_numeric(df_final['daytime'])

In [8]:
df_final['replies_time']=pd.to_numeric(df_final['replies_time'])

In [9]:
df_final["daytime"] = df_final["daytime"].apply(lambda x: round(x))

In [10]:
df_final["replies_time"] = df_final["replies_time"].apply(lambda x: round(x))

In [11]:
def time_conversor(x):
    return datetime.utcfromtimestamp(x).strftime('%d-%m-%Y %H:%M')

In [12]:
df_final["dayt"] = df_final["daytime"].apply(lambda x: time_conversor(x))

In [13]:
df_final["rep_time"] = df_final["replies_time"].apply(lambda x: time_conversor(x))

In [14]:
df_final["dayt"]=df_final["dayt"].apply(lambda x: datetime.strptime(x,'%d-%m-%Y %H:%M'))
df_final['date'] = pd.to_datetime(df_final['dayt']).dt.date
df_final['time'] = pd.to_datetime(df_final['dayt']).dt.time

In [15]:
df_final['rep_time']=df_final['rep_time'].astype(str)
#df_final["dayt"]=df_final["dayt"].apply(lambda x: datetime.strptime(x,'%H:%M'))
df_final['rep_time'] = pd.to_datetime(df_final['rep_time']).dt.time

In [16]:
df_final.drop(['dayt','daytime','replies_time'], axis=1, inplace=True)

In [17]:
users=json.load(open('slack/users.json'))

In [18]:
user_id=[]
name=[]
for user in users:
    user_id.append(user['id'])
    if 'real_name' in user.keys():
        name.append(user['real_name'])
    else:
        name.append(0)

#users=pd.DataFrame({'id':user_id,'name':name})
#users=users.drop_duplicates(subset="name",inplace=True)
users=dict(zip(user_id,name))

In [20]:
df=df_final

In [21]:
id_list=list(df['user'])
df['user']=[users[item] for item in id_list]


In [22]:
rea_list=list(df['reactions_user'])   

In [23]:
for i, item in enumerate(rea_list):
    if item != 0:
        rea_list[i]=users[item]
df['reactions_user']=rea_list

In [24]:
rep_list=list(df['replies_user'])
for i, item in enumerate(rep_list):
    if item!=0:
        rep_list[i]=users[item]
df['replies_user']=rep_list

In [25]:
df

,user,text,file_content,reactions,channel,reply_users,replies_user,file_url,link,reactions_user,rep_time,date,time
0,Sian Davies,<@U021UNQGZC4> has joined the channel,False,0,getting_started_tips,0,0,0,0,0,00:00:00,2021-05-12,09:43:00
1,Sian Davies,<@U021UNQGZC4> set the channel description: a ...,False,0,getting_started_tips,0,0,0,0,0,00:00:00,2021-05-12,09:43:00
2,Madini Liebscher,<@U0217P72YR5> has joined the channel,False,0,getting_started_tips,0,0,0,0,0,00:00:00,2021-05-12,09:43:00
3,miguel.simon,<@U021NHS652N> has joined the channel,False,0,getting_started_tips,0,0,0,0,0,00:00:00,2021-05-12,09:43:00
4,Jonas Tonndorff,<@U021TDD883V> has joined the channel,False,0,getting_started_tips,0,0,0,0,0,00:00:00,2021-05-13,09:52:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2608,Pedro Pablo Errazuriz,This week I'm working with the company of Seba...,False,2,music,0,0,0,0,Sian Davies,00:00:00,2021-06-25,08:31:00
2609,Uroš Petković,<https://soundcloud.com/crushedmusic>,False,0,music,0,0,0,0,0,00:00:00,2021-07-20,09:55:00
2610,Uroš Petković,<https://shazalakazoo.com/>,False,0,music,0,0,0,0,0,00:00:00,2021-07-20,09:58:00
2611,Uroš Petković,<https://open.spotify.com/artist/1YumFgivFXVVg...,False,0,music,0,0,0,0,0,00:00:00,2021-07-20,10:00:00


In [ ]:
df_channel=df.groupby(['channel']).count()

In [ ]:
df_channel

In [ ]:
df_user=df.groupby(['user','text']).sum()

In [ ]:
df_user

In [ ]:
df['user'].nunique()

In [ ]:
df.to_csv('df.csv')